In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
import torchvision.transforms as T

device = ('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.empty_cache()

/tmp/ipykernel_120387/1264752695.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:


class BirdsDataset(Dataset):
    initial_path = "./100-birds/"
    transform = T.Resize((190, 190))

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        file_url = self.x[idx]
        image = read_image(f"{self.initial_path}{file_url}")
        image = self.transform(image)

        image = image.type(torch.float32)
        
        return image/255, int(self.y[idx])

In [8]:
class BirdModel(nn.Module):
    def __init__(self):
        super(BirdModel, self).__init__()
        
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(3, 96, (11,11), stride=4),
            nn.MaxPool2d((3,3)),
            nn.Conv2d(96, 256, (3,3), padding=2),
            nn.MaxPool2d((3,3), stride=2),
            nn.Conv2d(256, 256, (3,3), padding=2),
            nn.MaxPool2d((3,3), stride=2),
            nn.Flatten(),

            nn.Linear(4096, 3000),
            nn.ReLU(),
            nn.Linear(3000, 600),
            nn.ReLU(),
            nn.Linear(600, 525),
            


        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:

            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):

        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)

        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss

In [10]:
def main():
  train_data = pd.read_csv('./100-birds/birds_train.csv', usecols=['filepaths', 'class id'])
  test_data = pd.read_csv('./100-birds/birds_test.csv', usecols=['filepaths','class id'])

  batch_size = 50
  epochs = 20
  learning_rate = 0.0003

  loss_fn = nn.CrossEntropyLoss().to(device)
  model = BirdModel().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  
  train_dataset = BirdsDataset(train_data['filepaths'].values, train_data['class id'].values)
  test_dataset = BirdsDataset(test_data['filepaths'].values, test_data['class id'].values)



  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)


main()


Epoch 1
-------------------------------
loss: 6.259929  [    0/84635]
loss: 6.183586  [ 5000/84635]
loss: 5.960428  [10000/84635]
loss: 5.806148  [15000/84635]
loss: 5.402849  [20000/84635]
loss: 4.779073  [25000/84635]
loss: 4.855789  [30000/84635]
loss: 4.624241  [35000/84635]
loss: 4.327066  [40000/84635]
loss: 4.409547  [45000/84635]
loss: 4.556392  [50000/84635]
loss: 4.057748  [55000/84635]
loss: 3.919068  [60000/84635]
loss: 3.748756  [65000/84635]
loss: 3.187983  [70000/84635]
loss: 3.758808  [75000/84635]
loss: 3.430858  [80000/84635]
Test Error: 
 Accuracy: 39.6%, Avg loss: 2.588059 

Epoch 2
-------------------------------
loss: 2.546728  [    0/84635]
loss: 3.004635  [ 5000/84635]
loss: 2.576966  [10000/84635]
loss: 2.478292  [15000/84635]
loss: 2.959817  [20000/84635]
loss: 2.661288  [25000/84635]
loss: 2.053031  [30000/84635]
loss: 3.186003  [35000/84635]
loss: 2.205069  [40000/84635]
loss: 2.475085  [45000/84635]
loss: 2.427556  [50000/84635]
loss: 1.660644  [55000/84635